In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [23]:
spark = SparkSession.builder.appName("SparkDev-streaming tutorial").master("local[*]").getOrCreate()

In [24]:
help(spark.readStream.format)

Help on method format in module pyspark.sql.streaming:

format(source: str) -> 'DataStreamReader' method of pyspark.sql.streaming.DataStreamReader instance
    Specifies the input data source format.
    
    .. versionadded:: 2.0.0
    
    Parameters
    ----------
    source : str
        name of the data source, e.g. 'json', 'parquet'.
    
    Notes
    -----
    This API is evolving.
    
    Examples
    --------
    >>> s = spark.readStream.format("text")



In [25]:
input_table = spark.readStream.format("socket").option("host", "localhost").option("port", "9999").load()

23/03/26 08:50:04 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [26]:
print(input_table.isStreaming)
input_table.printSchema()

True
root
 |-- value: string (nullable = true)



In [27]:
words = input_table.select(
    explode(
        split(input_table.value, " ")
    ).alias("word")
)

wordCount = words.groupBy("word").count()

console_writer = wordCount.writeStream.outputMode("update").format("console").start()

23/03/26 08:50:05 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/1x/36nz44_569501n4xs0px2_8m0000gn/T/temporary-4be3fb10-c80d-4d20-bbf2-21732d8cabb1. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/03/26 08:50:05 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [28]:
console_writer.awaitTermination()
print("Terminated")

-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+---------+-----+
|     word|count|
+---------+-----+
|streaming|    1|
|    spark|    1|
+---------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+----------+-----+
|      word|count|
+----------+-----+
| streaming|    2|
|structured|    1|
+----------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+----------+-----+
|      word|count|
+----------+-----+
| streaming|    3|
|     spark|    2|
|structured|    2|
+----------+-----+

23/03/26 08:51:08 WARN TextSocketMicroBatchStream: Stream closed by localhost:9999


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/pavanmantha/Library/Application Support/JetBrains/DataSpell2022.3/projects/workspace/venv/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/pavanmantha/Library/Application Support/JetBrains/DataSpell2022.3/projects/workspace/venv/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 